# AmpForm demo PANDA Seminar December 2021

This notebook accompanies [these slides](https://docs.google.com/presentation/d/e/2PACX-1vSymz5AjdhPw4Kz1pKhdFMnFGYuQvVaC8WbV_HTg770x6RDYoP-Anv9tn88DSuzvSiiQ9F4pcDGVExv/pub). They were presented during a PANDA Seminar on 13 December 2021.

Related notebooks for this presentation:
- [QRules demo](./qrules.ipynb)
- [TensorWaves demo](./tensorwaves.ipynb)

For more extensive examples, see **[ampform.rtfd.io](https://ampform.readthedocs.io)**.

## Install dependencies

In [ ]:
%pip install -q ampform[viz]==0.12.*

In [ ]:
%load_ext autoreload
%autoreload
from functools import lru_cache

import ampform
import graphviz
import ipywidgets
import qrules
import symplot
import sympy as sp
from ampform.dynamics import BlattWeisskopfSquared, EnergyDependentWidth
from ampform.dynamics.builder import (
    create_analytic_breit_wigner,
    create_relativistic_breit_wigner_with_ff,
)
from ampform.dynamics.kmatrix import NonRelativisticKMatrix, RelativisticKMatrix
from ampform.sympy import UnevaluatedExpression
from IPython.display import Math, display
from ipywidgets import Checkbox, SelectionSlider, SelectMultiple, ToggleButtons
from sympy.physics.quantum.spin import WignerD


class Symbol(sp.Symbol):
    pass

## $K$-matrix expressions

In [ ]:
n = sp.Symbol("n_R")
matrix = RelativisticKMatrix.formulate(
    n_channels=1,
    n_poles=n,
)
matrix[0, 0]

In [ ]:
matrix = NonRelativisticKMatrix.formulate(
    n_channels=2,
    n_poles=1,
).doit()
matrix[0, 0].simplify()

## Example $D^0 \to K^0 K^- K^+$

### Generate transitions

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="D0",
    final_state=["K0", "K-", "K+"],
    allowed_intermediate_particles=["a(0)(980)0", "a(0)(1450)0", "phi(1020)"],
    formalism="helicity",
)

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True, render_final_state_id=True)
graphviz.Source(dot)

Note that one of the resonances, $a_0(980)$, lies **below threshold**, which means we should parametrize it with an analytic continuation.

In [ ]:
PDG = qrules.load_pdg()
PDG["a(0)(980)0"].mass < PDG["K-"].mass + PDG["K-"].mass

In [ ]:
builder = ampform.get_builder(reaction)
resonances = reaction.get_intermediate_particles()
for p in resonances:
    builder.set_dynamics(p.name, create_relativistic_breit_wigner_with_ff)
builder.set_dynamics("a(0)(980)0", create_analytic_breit_wigner)
model = builder.formulate()

In [ ]:
amplitude_expressions = {
    expr: sp.Symbol(name)
    for name, expr in model.components.items()
    if name.startswith("A")
}
top_expr = model.expression.xreplace(amplitude_expressions)
top_expr

### Examine one of the amplitudes

In [ ]:
some_amplitude = model.components[
    R"A_{D^{0}_{0} \to K^{0}_{0} \phi(1020)_{0}; \phi(1020)_{0} \to K^{+}_{0} K^{-}_{0}}"
]

In [ ]:
def round_nested(expression, n_decimals):
    for node in sp.preorder_traversal(expression):
        if node.free_symbols:
            continue
        if isinstance(node, (float, sp.Float)):
            expression = expression.xreplace({node: node.n(n_decimals)})
    return expression


A_step1 = some_amplitude
A_step2 = symplot.partial_doit(A_step1, (BlattWeisskopfSquared, WignerD))
A_step2 = symplot.partial_doit(A_step2, EnergyDependentWidth)
mass_substitutions = {
    sp.Symbol(f"m_{i}", real=True): p.mass for i, p in reaction.final_state.items()
}
substitutions = {
    sp.sqrt(2): sp.sqrt(2).n(),
    **mass_substitutions,
    **model.parameter_defaults,
}
A_step3 = some_amplitude.doit().xreplace(substitutions)
A_step3 = round_nested(A_step3, n_decimals=2)
with sp.assuming(*map(sp.Q.positive, A_step3.free_symbols)):
    A_step3 = A_step3.refine()
display(
    A_step1,
    Math("=" + sp.latex(A_step2)),
    Math("=" + sp.latex(A_step3)),
)

### Visualize expression tree
<!-- cspell:ignore bgcolor dodgerblue fillcolor indianred -->

In [ ]:
dot_style = [
    (sp.Basic, {"style": "filled", "fillcolor": "white"}),
    (sp.Atom, {"color": "gray", "style": "filled", "fillcolor": "white"}),
    (sp.Symbol, {"color": "dodgerblue1"}),
    ((UnevaluatedExpression, WignerD), {"color": "indianred2"}),
]
dot = sp.dotprint(A_step3, styles=dot_style, size=10)
graphviz.Source(dot)

In [ ]:
aliases = ["phi", "a(0)(980)", "a(0)(1450)", "full model"]
amplitudes = dict(zip(aliases, model.components.values()))


def partial_doit(expr, classes):
    for sub_expr in sp.preorder_traversal(expr):
        if type(sub_expr).__name__ in classes:
            new_sub_expr = sub_expr.doit(deep=False)
            new_sub_expr = partial_doit(new_sub_expr, classes)
            expr = expr.xreplace({sub_expr: new_sub_expr})
    return expr


@lru_cache(maxsize=None)
def get_amplitude(
    name,
    doit,
    substitute_masses,
    substitute_parameters,
    round_floats,
    unfolded_classes,
):
    expr = amplitudes[name]
    if len(unfolded_classes):
        expr = partial_doit(expr, unfolded_classes)
    if doit:
        expr = expr.doit()
    if substitute_masses:
        expr = expr.subs(mass_substitutions)
    if substitute_parameters:
        expr = expr.subs(model.parameter_defaults)
    if round_floats:
        expr = round_nested(expr, n_decimals=2)
    return expr


@lru_cache(maxsize=None)
def create_graph(expression, fig_size, visualize_cse):
    if fig_size == "full":
        fig_size = None
    dot = sp.dotprint(
        expression, styles=dot_style, size=fig_size, repeat=not visualize_cse
    )
    return graphviz.Source(dot)


@ipywidgets.interact(
    amplitude=ToggleButtons(
        description="Amplitude:", value=aliases[2], options=list(amplitudes)
    ),
    rendering=ToggleButtons(
        description="Rendering:", options=["tree", "math", "unicode"]
    ),
    doit=Checkbox(description="Unfold all nodes"),
    substitute_parameters=Checkbox(description="Substitute parameters"),
    substitute_masses=Checkbox(description="Substitute stable masses"),
    visualize_cse=Checkbox(description="Common sub-expressions"),
    round_floats=Checkbox(description="Round floats", value=True),
    fig_size=SelectionSlider(
        description="Figure size", options=["full", 5, 7, 10, 15, 20, 50]
    ),
    unfolded_classes=SelectMultiple(
        description="Unfold:",
        options=[
            "BlattWeisskopfSquared",
            "BreakupMomentumSquared",
            "EnergyDependentWidth",
            "PhaseSpaceFactor",
            "PhaseSpaceFactorAbs",
            "PhaseSpaceFactorAnalytic",
            "WignerD",
        ],
        rows=7,
    ),
)
def visualize(
    rendering,
    amplitude,
    substitute_masses,
    substitute_parameters,
    round_floats,
    doit,
    visualize_cse,
    unfolded_classes,
    fig_size,
):
    expression = get_amplitude(
        amplitude,
        doit,
        substitute_masses,
        substitute_parameters,
        round_floats,
        unfolded_classes,
    )
    if rendering == "unicode":
        print(sp.pretty(expression))
    elif rendering == "math":
        display(expression)
    else:
        graph = create_graph(expression, fig_size, visualize_cse)
        display(graph)

## Expression manipulation

In [ ]:
expr = amplitudes["a(0)(1450)"].doit()
graph_style = dict(
    bgcolor="transparent",
    size=15,
    styles=[
        *dot_style,
        (Symbol, {"fillcolor": "lightblue", "style": "filled"}),
    ],
)
dot = sp.dotprint(expr, **graph_style)
display(expr)
graphviz.Source(dot)

In [ ]:
mass_symbols = {s: Symbol(s.name, **s.assumptions0) for s in mass_substitutions}
colored_symbols = {
    s: sp.Symbol(Rf"\color{{RoyalBlue}}{{{s.name}}}", **s.assumptions0)
    for s in mass_substitutions
}
dot = sp.dotprint(expr.xreplace(mass_symbols), **graph_style)
display(expr.xreplace(colored_symbols))
graphviz.Source(dot)

In [ ]:
expr_subs = expr.subs(mass_substitutions)
expr_subs = round_nested(expr_subs, n_decimals=2)
node = next(s for s in sp.preorder_traversal(expr_subs) if isinstance(s, sp.Float))
dot = sp.dotprint(expr_subs.subs(node, Symbol(f"{node}")), **graph_style)
display(expr_subs.subs(node, sp.Symbol(Rf"\color{{RoyalBlue}}{node}")))
graphviz.Source(dot)

## Symbolic kinematic variables

In [ ]:
kinematic_variables = sorted(A_step3.free_symbols, key=str)
kinematic_variables

In [ ]:
theta_1_12 = kinematic_variables[1]
model.kinematic_variables[theta_1_12]

In [ ]:
model.kinematic_variables[theta_1_12].doit()